In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
dataset = pd.read_csv("heart_failure_clinical_records.csv")

In [3]:
dataset.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,55.0,0,748,0,45,0,263358.03,1.3,137,1,1,88,0
1,65.0,0,56,0,25,0,305000.00,5.0,130,1,0,207,0
2,45.0,0,582,1,38,0,319000.00,0.9,140,0,0,244,0
3,60.0,1,754,1,40,1,328000.00,1.2,126,1,0,90,0
4,95.0,1,582,0,30,0,461000.00,2.0,132,1,0,50,1


In [4]:
#DATASET 
dataset.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,60.288736,0.474400,586.760600,0.439400,37.734600,0.364800,265075.404370,1.369106,136.808200,0.645600,0.311800,130.678800,0.313600
std,11.697243,0.499394,976.733979,0.496364,11.514855,0.481422,97999.758622,1.009750,4.464236,0.478379,0.463275,77.325928,0.464002
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.500000,113.000000,0.000000,0.000000,4.000000,0.000000
25%,50.000000,0.000000,121.000000,0.000000,30.000000,0.000000,215000.000000,0.900000,134.000000,0.000000,0.000000,74.000000,0.000000
50%,60.000000,0.000000,248.000000,0.000000,38.000000,0.000000,263358.030000,1.100000,137.000000,1.000000,0.000000,113.000000,0.000000
75%,68.000000,1.000000,582.000000,1.000000,45.000000,1.000000,310000.000000,1.400000,140.000000,1.000000,1.000000,201.000000,1.000000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.400000,148.000000,1.000000,1.000000,285.000000,1.000000


DEPENDENT OR PREDICTED VARIABLE IS "DEATH_EVENT"

In [5]:
#CREATE A TREE DATA STRUCTURE TO STORE THE DECISION TREE
class Tree:
    def __init__(self ,feature = None ,threshold = 0 ,left = None ,right = None, value = None ):
        self.left = left
        self.right = right
        self.value = value
        self.feature = feature
        self.threshold = threshold
        
    def print_tree(self):
        if self.left is not None:
            self.left.print_tree()
        if self.right is not None:
            self.right.print_tree()
        print("feature = ",self.feature, "and the split value is = ",self.threshold," The value is ",self.value)
        

In [6]:
#Split the dataset into training and testing dataset
#training dataset is 80% and testing dataset is 20%
train_size = 0.8
split_index = int(train_size * len(dataset))

# Use iloc to create training and testing sets
train_df = dataset.iloc[:split_index]
test_df = dataset.iloc[split_index:]

In [7]:
#split the left subset of the dataset based on condition dataset[feature] < threshold
def split_left(dataset,feature,threshold):
    return dataset[dataset[feature] < threshold]

In [8]:
#split the right subset of the dataset based on condition dataset[feature] >= threshold
def split_right(dataset,feature,threshold):
    return dataset[dataset[feature] >= threshold]

In [9]:
#best fit returns the list of best fitting thresholds of a feature
def best_fit(variables):

    uniqueValues = list(set(variables))
    uniqueValues.sort()
#     print(uniqueValues)
    thresholds = []
    for i in range(0,len(uniqueValues)-1):
        temp = uniqueValues[i] + uniqueValues[i+1]
        temp = temp/2
        thresholds.append(temp)
    
    return thresholds


In [10]:
#find which value is dominant or most occured in the predicted or dependent column.
def dominant_class(feature):
    frequency = {} 
    
    for value in feature:
        if value in frequency:
            frequency[value] += 1
        else:
            frequency[value] = 1
    
#     print(frequency)
    
    maximum = 0
    values = ""
    
    for key,value in frequency.items():
        if(frequency[key] > maximum):
            maximum = frequency[key]
            values = key
    
    return values
    

In [11]:
#entropy calculation 
def entropy(feature):
    total = len(feature)
    
    frequency = {}
    
    for value in feature:
        if value in frequency:
            frequency[value] += 1
        else:
            frequency[value] = 1
        
    result = 0
    
    for value,count in frequency.items():
        prob = count/total
        if(prob != 0):
            result += -(prob) * math.log(prob,2)
    
    return result


In [12]:
#FUNCTION TO GET RANDOM FEATURE SUBSET
def get_random_feature_subsets(features, subset_size):
    feature_subsets = []
    subset = np.random.choice(features, subset_size, replace=False)
    return subset

In [13]:
#FUNCTION TO CREATE A BOOTSTRAP SAMPLING
def bootstrap_sampling(dataset):
    # Generate random indices for sampling
    indices = np.random.choice(dataset.index, size=len(dataset), replace=True)
    
    #create a bootstrap sample dataset using the random indices
    sample_dataset = dataset.iloc[indices]
    return sample_dataset

In [14]:
#EXAMPLE DATASET CREATED USING BOOTSTRAP DATA SAMPLING METHOD
bootstrap_sampling(train_df)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
1026,55.0,1,180,1,45,0,263358.03,1.18,137,1,1,211,0
2769,90.0,1,60,1,50,0,226000.00,1.00,134,1,0,30,1
175,53.0,1,1071,0,45,0,305000.00,1.10,137,1,1,209,0
353,65.0,1,305,0,50,0,263358.03,1.18,137,1,1,87,0
3741,60.0,1,607,0,40,0,216000.00,0.60,138,1,1,54,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3151,50.0,1,115,0,20,0,189000.00,0.80,139,1,0,146,0
976,50.0,1,159,1,30,0,300000.00,1.20,138,0,0,29,0
1073,87.0,1,149,1,20,0,70000.00,2.40,134,1,0,140,1
3484,64.0,0,143,0,25,0,246000.00,2.40,135,1,0,214,0


In [15]:
#STORE ALL THE INDEPENDENT OR PREDICTOR FEATURES IN A LIST
features = list(dataset.columns)
features.remove("DEATH_EVENT")
print(features)

['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']


In [16]:
#fit or train the model using decision tree
def decision_tree_fit(parent,depth,maximum_depth = 100,max_leaf = 5):
    
    #if the node is at the maximum depth it should stop growing.
    #if the node is still heterogeneous(more than one species) the result at this node is most dominant class in that node.
    #That is the species  with most frequency in this node. 
    if(depth == maximum_depth):  
        death_event = dominant_class(parent["DEATH_EVENT"])
        print("maximum depth attained death event = ",death_event)
        return Tree(value = death_event)
    
    if(len(parent) == max_leaf):
        death_event = dominant_class(parent["DEATH_EVENT"])
        print("max leaf size attained death event = ",death_event)
        return Tree(value = death_event)

    
    parent_entropy = entropy(parent["DEATH_EVENT"])
        
    maximum_information_gain = 0
    maximum_information_gain_threshold = 0
    maximum_information_gain_feature = ""
    left_node = None
    right_node = None
    left_node_entropy = 0
    right_node_entropy = 0
    left_subtree = None
    right_subtree = None
    
    for feature in features:
                
        thresholds = best_fit(parent[feature])
        #print(thresholds)
                
        for threshold in thresholds:
            left = split_left(parent,feature,threshold)
            right = split_right(parent,feature,threshold)
            
            #child entropy calculation
            left_entropy = (len(left)/len(parent)) * entropy(left["DEATH_EVENT"])
            right_entropy = (len(right)/len(parent)) * entropy(right["DEATH_EVENT"])
            child_entropy = left_entropy + right_entropy
            
            #print("child entropy ",child_entropy)
            
            information_gain = parent_entropy - child_entropy
            #print("information gain ",information_gain)
            
            if(information_gain > maximum_information_gain):
                maximum_information_gain = information_gain
                max_ig_threshold = threshold #maximum information gained threshold
                max_ig_feature = feature #maximum information gained feature
                left_node = left
                right_node = right
                left_node_entropy = left_entropy
                right_node_entropy = right_entropy
        
    if(maximum_information_gain == 0):
        return Tree(value=parent['DEATH_EVENT'].iloc[0])
        
    elif(maximum_information_gain != 0):
        
        print()
        print(f"FEATURE : {max_ig_feature}")
        print(f"MAXIMUM INFORMATION GAINED : {maximum_information_gain}")
        print(f"SPLITTED AS {max_ig_feature} < {max_ig_threshold} AND {max_ig_feature} >= {max_ig_threshold}")
        print(f"LEFT NODE WITH ENTROPY = ",entropy(left_node["DEATH_EVENT"]))
#         print(left_node)
        print(f"RIGHT NODE WITH ENTROPY = ",entropy(right_node["DEATH_EVENT"]))
#         print(right_node)
            
        if left_node is not None :
            left_subtree = decision_tree_fit(left_node, depth + 1, maximum_depth,max_leaf)
                    
        if right_node is not None :
            right_subtree = decision_tree_fit(right_node, depth + 1, maximum_depth,max_leaf)
        
        
    return Tree(max_ig_feature,max_ig_threshold,left_subtree,right_subtree)
        
          
        

In [17]:
#fit or train the model using random forest
def random_forest_fit(parent,depth,maximum_depth = 100,max_leaf = 5):
    
    #if the node is at the maximum depth it should stop growing.
    #if the node is still heterogeneous(more than one species) the result at this node is most dominant class in that node.
    #That is the species  with most frequency in this node. 
    if(depth == maximum_depth):  
        death_event = dominant_class(parent["DEATH_EVENT"])
        print("maximum depth attained death event = ",death_event)
        return Tree(value = death_event)
    
    if(len(parent) == max_leaf):
        death_event = dominant_class(parent["DEATH_EVENT"])
        print("max leaf size attained death event = ",death_event)
        return Tree(value = death_event)

    
    parent_entropy = entropy(parent["DEATH_EVENT"])
        
    maximum_information_gain = 0
    maximum_information_gain_threshold = 0
    maximum_information_gain_feature = ""
    left_node = None
    right_node = None
    left_node_entropy = 0
    right_node_entropy = 0
    left_subtree = None
    right_subtree = None
    
    random_subset = get_random_feature_subsets(features,(int(math.sqrt(len(features)))))
#     print("random subset = ",random_subset)
    for feature in random_subset:
                
        thresholds = best_fit(parent[feature])
        #print(thresholds)
                
        for threshold in thresholds:
            left = split_left(parent,feature,threshold)
            right = split_right(parent,feature,threshold)
            
            #child entropy calculation
            left_entropy = (len(left)/len(parent)) * entropy(left["DEATH_EVENT"])
            right_entropy = (len(right)/len(parent)) * entropy(right["DEATH_EVENT"])
            child_entropy = left_entropy + right_entropy
            
            #print("child entropy ",child_entropy)
            
            information_gain = parent_entropy - child_entropy
            #print("information gain ",information_gain)
            
            if(information_gain > maximum_information_gain):
                maximum_information_gain = information_gain
                max_ig_threshold = threshold #maximum information gained threshold
                max_ig_feature = feature #maximum information gained feature
                left_node = left
                right_node = right
                left_node_entropy = left_entropy
                right_node_entropy = right_entropy
        
    if(maximum_information_gain == 0):
        return Tree(value=parent['DEATH_EVENT'].iloc[0])
        
    elif(maximum_information_gain != 0):
        
        print()
        print(f"FEATURE : {max_ig_feature}")
        print(f"MAXIMUM INFORMATION GAINED : {maximum_information_gain}")
        print(f"SPLITTED AS {max_ig_feature} < {max_ig_threshold} AND {max_ig_feature} >= {max_ig_threshold}")
        print(f"LEFT NODE WITH ENTROPY = ",entropy(left_node["DEATH_EVENT"]))
#         print(left_node)
        print(f"RIGHT NODE WITH ENTROPY = ",entropy(right_node["DEATH_EVENT"]))
#         print(right_node)
            
        if left_node is not None :
            left_subtree = random_forest_fit(left_node, depth + 1, maximum_depth,max_leaf)
                    
        if right_node is not None :
            right_subtree = random_forest_fit(right_node, depth + 1, maximum_depth,max_leaf)
        
        
    return Tree(max_ig_feature,max_ig_threshold,left_subtree,right_subtree)
        
          
        

In [18]:
def Random_Forest(dataset,maximum_trees=100,maximum_depth=100,maximum_leaf=5):
    
    trees = []
    
    for i in range(0,maximum_trees):
        #create a bootstrap sample dataset
        sample_dataset = bootstrap_sampling(dataset)
#         print(len(sample_dataset))
        #build trees and store it in a list
        tree = random_forest_fit(sample_dataset,0,maximum_depth,maximum_leaf)
        trees.append(tree)
        
        print()
        print(i,"th TREE IS BUILT")
        print()
    
    print("TOTAL TREES BUILT : ",len(trees))
    return trees

In [19]:
#predict a test dataset with trained model by traversing the trained random forest model
def predict_using_random_forest(dataset,trees):
    values = []
    for tree in trees:

        while(tree.left != None and tree.right != None ):
            feature = tree.feature
            threshold = tree.threshold

            if(dataset[feature] < threshold):
                tree = tree.left
            else:
                tree = tree.right 
        values.append(tree.value)

#     print("values ",values)
    return dominant_class(values)
    

In [20]:
#predict a test dataset with trained model by traversing the trained decision tree model
def predict_using_decision_tree(dataset,tree):
    while(tree.left != None and tree.right != None ):
        feature = tree.feature
        threshold = tree.threshold
        if(dataset[feature] < threshold):
            tree = tree.left
        else:
            tree = tree.right 
        
    return tree.value
    

In [21]:
#function to calculate the accuracy of the trained model
def accuracy(predictions,actual_values):
    #print(actual_values)
    total_predictions = len(predictions)
    correct_predictions = 0
    print("predictions actual values\n")
    for i in range(0,total_predictions):
        print(predictions[i], " ", actual_values[i])
        if(predictions[i] == actual_values[i]):
            correct_predictions = correct_predictions + 1
    
    return (correct_predictions/total_predictions) * 100
        

In [22]:
#test dataset
test_df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
4000,50.0,1,121,1,40,0,260000.0,0.7,130,1,0,175,0
4001,53.0,1,1808,0,60,1,249000.0,0.7,138,1,1,106,0
4002,68.0,1,1021,1,35,0,271000.0,1.1,134,1,0,197,0
4003,53.0,1,1808,0,60,1,249000.0,0.7,138,1,1,106,0
4004,52.0,1,191,1,30,1,334000.0,1.0,142,1,1,216,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,45.0,0,582,1,55,0,543000.0,1.0,132,0,0,250,0
4996,60.0,1,582,0,30,1,127000.0,0.9,145,0,0,95,0
4997,95.0,1,112,0,40,1,196000.0,1.0,138,0,0,24,1
4998,65.0,1,160,1,20,0,327000.0,2.7,116,0,0,8,1


PREDICTING WITH RANDOM FOREST MODEL

In [23]:
#create a random forest model with 100 trees with maximum depth of 100 and maximum leaf size of 10
heart_disease_decision_tree_model = Random_Forest(train_df,100,100,10)


FEATURE : time
MAXIMUM INFORMATION GAINED : 0.2856466961537766
SPLITTED AS time < 67.5 AND time >= 67.5
LEFT NODE WITH ENTROPY =  0.5354236690186608
RIGHT NODE WITH ENTROPY =  0.6372649975612115

FEATURE : sex
MAXIMUM INFORMATION GAINED : 0.015028508276259789
SPLITTED AS sex < 0.5 AND sex >= 0.5
LEFT NODE WITH ENTROPY =  0.7100504703437547
RIGHT NODE WITH ENTROPY =  0.433376842219427

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.11582974517981504
SPLITTED AS ejection_fraction < 32.5 AND ejection_fraction >= 32.5
LEFT NODE WITH ENTROPY =  0.9206092191136652
RIGHT NODE WITH ENTROPY =  0.2351933818192415

FEATURE : serum_sodium
MAXIMUM INFORMATION GAINED : 0.37742007536280164
SPLITTED AS serum_sodium < 136.5 AND serum_sodium >= 136.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.9709505944546686

FEATURE : serum_sodium
MAXIMUM INFORMATION GAINED : 0.8829183153667972
SPLITTED AS serum_sodium < 138.5 AND serum_sodium >= 138.5
LEFT NODE WITH ENTROPY =  0.143726169


FEATURE : serum_creatinine
MAXIMUM INFORMATION GAINED : 0.07088001952259809
SPLITTED AS serum_creatinine < 0.6499999999999999 AND serum_creatinine >= 0.6499999999999999
LEFT NODE WITH ENTROPY =  0.6193821946787638
RIGHT NODE WITH ENTROPY =  0.28180711010244014

FEATURE : high_blood_pressure
MAXIMUM INFORMATION GAINED : 0.6193821946787638
SPLITTED AS high_blood_pressure < 0.5 AND high_blood_pressure >= 0.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : diabetes
MAXIMUM INFORMATION GAINED : 0.020661249220004396
SPLITTED AS diabetes < 0.5 AND diabetes >= 0.5
LEFT NODE WITH ENTROPY =  0.11292981320626021
RIGHT NODE WITH ENTROPY =  0.42080948947668984

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.01585328352007563
SPLITTED AS age < 43.5 AND age >= 43.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.19942077803751096

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.1091277114280157
SPLITTED AS age < 44.5 AND age >= 44.5
LEFT NODE WITH ENTROPY =  0.811278124


FEATURE : platelets
MAXIMUM INFORMATION GAINED : 0.03152761979892534
SPLITTED AS platelets < 98500.0 AND platelets >= 98500.0
LEFT NODE WITH ENTROPY =  0.5225593745369408
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.5225593745369408
SPLITTED AS time < 248.0 AND time >= 248.0
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.268276207774263
SPLITTED AS time < 73.0 AND time >= 73.0
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.9477073729342066

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.26212022550395353
SPLITTED AS ejection_fraction < 55.0 AND ejection_fraction >= 55.0
LEFT NODE WITH ENTROPY =  0.8112781244591328
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.3652938975319328
SPLITTED AS age < 80.5 AND age >= 80.5
LEFT NODE WITH ENTROPY =  0.9910760598382222
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : ejection_fraction
MAXIMUM INFORMATION G


FEATURE : age
MAXIMUM INFORMATION GAINED : 0.6962122601251458
SPLITTED AS age < 55.0 AND age >= 55.0
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.7706290693639405
SPLITTED AS age < 77.5 AND age >= 77.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

0 th TREE IS BUILT


FEATURE : time
MAXIMUM INFORMATION GAINED : 0.29462710923153135
SPLITTED AS time < 73.5 AND time >= 73.5
LEFT NODE WITH ENTROPY =  0.6488376217482742
RIGHT NODE WITH ENTROPY =  0.5810550228646842

FEATURE : serum_sodium
MAXIMUM INFORMATION GAINED : 0.07646187429425699
SPLITTED AS serum_sodium < 136.5 AND serum_sodium >= 136.5
LEFT NODE WITH ENTROPY =  0.2833361520889187
RIGHT NODE WITH ENTROPY =  0.8573196400507843

FEATURE : smoking
MAXIMUM INFORMATION GAINED : 0.04508727211503874
SPLITTED AS smoking < 0.5 AND smoking >= 0.5
LEFT NODE WITH ENTROPY =  0.4394969869215134
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : creatinine_phosphokinase
MAXIMUM 


FEATURE : age
MAXIMUM INFORMATION GAINED : 0.02782668714688996
SPLITTED AS age < 71.0 AND age >= 71.0
LEFT NODE WITH ENTROPY =  0.5013700758412414
RIGHT NODE WITH ENTROPY =  0.9279643171690706

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.08936381135009175
SPLITTED AS ejection_fraction < 32.5 AND ejection_fraction >= 32.5
LEFT NODE WITH ENTROPY =  0.892940459538831
RIGHT NODE WITH ENTROPY =  0.24154235674697647

FEATURE : serum_creatinine
MAXIMUM INFORMATION GAINED : 0.15223812817563698
SPLITTED AS serum_creatinine < 0.95 AND serum_creatinine >= 0.95
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.9754220454763884

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.013450685721293376
SPLITTED AS ejection_fraction < 22.5 AND ejection_fraction >= 22.5
LEFT NODE WITH ENTROPY =  0.9918207974218424
RIGHT NODE WITH ENTROPY =  0.9554913432639305

FEATURE : platelets
MAXIMUM INFORMATION GAINED : 0.6135947730809936
SPLITTED AS platelets < 275000.0 AND platelets

...


FEATURE : age
MAXIMUM INFORMATION GAINED : 0.10250294121125214
SPLITTED AS age < 58.5 AND age >= 58.5
LEFT NODE WITH ENTROPY =  0.6840384356390417
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : creatinine_phosphokinase
MAXIMUM INFORMATION GAINED : 0.09401798148694507
SPLITTED AS creatinine_phosphokinase < 145.5 AND creatinine_phosphokinase >= 145.5
LEFT NODE WITH ENTROPY =  0.8112781244591328
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : platelets
MAXIMUM INFORMATION GAINED : 0.8112781244591328
SPLITTED AS platelets < 417500.0 AND platelets >= 417500.0
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.01665049615838495
SPLITTED AS time < 232.5 AND time >= 232.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.24229218908241482

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.24229218908241482
SPLITTED AS time < 240.0 AND time >= 240.0
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : platelets
MAXIMUM INFOR


FEATURE : serum_creatinine
MAXIMUM INFORMATION GAINED : 0.05705497188034914
SPLITTED AS serum_creatinine < 1.815 AND serum_creatinine >= 1.815
LEFT NODE WITH ENTROPY =  0.9999435071707974
RIGHT NODE WITH ENTROPY =  0.739480936716638

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.3339823642934825
SPLITTED AS time < 210.0 AND time >= 210.0
LEFT NODE WITH ENTROPY =  0.8701883352082784
RIGHT NODE WITH ENTROPY =  0.19143325481419346

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.2636204818730754
SPLITTED AS ejection_fraction < 32.5 AND ejection_fraction >= 32.5
LEFT NODE WITH ENTROPY =  0.5149459065452171
RIGHT NODE WITH ENTROPY =  0.8366407419411673

FEATURE : platelets
MAXIMUM INFORMATION GAINED : 0.09685288486402965
SPLITTED AS platelets < 274500.0 AND platelets >= 274500.0
LEFT NODE WITH ENTROPY =  0.7158259310602149
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : platelets
MAXIMUM INFORMATION GAINED : 0.2917898580879097
SPLITTED AS platelets < 248000.0 AND platelets >= 248000.0


FEATURE : smoking
MAXIMUM INFORMATION GAINED : 0.08212210284660026
SPLITTED AS smoking < 0.5 AND smoking >= 0.5
LEFT NODE WITH ENTROPY =  0.6235702082127974
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : sex
MAXIMUM INFORMATION GAINED : 0.1187749307715592
SPLITTED AS sex < 0.5 AND sex >= 0.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.8112781244591328

FEATURE : serum_sodium
MAXIMUM INFORMATION GAINED : 0.46116172763922814
SPLITTED AS serum_sodium < 139.0 AND serum_sodium >= 139.0
LEFT NODE WITH ENTROPY =  0.4262286569981449
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.12719211097303257
SPLITTED AS time < 209.5 AND time >= 209.5
LEFT NODE WITH ENTROPY =  0.7642045065086203
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.7642045065086203
SPLITTED AS age < 58.5 AND age >= 58.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : serum_sodium
MAXIMUM INFORMATION GAINED : 0.10139880827199904
SPLITTE


FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.10629429292409986
SPLITTED AS ejection_fraction < 22.5 AND ejection_fraction >= 22.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.9380205673779887

FEATURE : creatinine_phosphokinase
MAXIMUM INFORMATION GAINED : 0.08573171349400721
SPLITTED AS creatinine_phosphokinase < 613.5 AND creatinine_phosphokinase >= 613.5
LEFT NODE WITH ENTROPY =  0.8486613058570995
RIGHT NODE WITH ENTROPY =  0.868533960265235

FEATURE : serum_sodium
MAXIMUM INFORMATION GAINED : 0.11713251597777308
SPLITTED AS serum_sodium < 135.5 AND serum_sodium >= 135.5
LEFT NODE WITH ENTROPY =  0.6733696130414387
RIGHT NODE WITH ENTROPY =  0.9456603046006402

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.4850288027999223
SPLITTED AS time < 143.5 AND time >= 143.5
LEFT NODE WITH ENTROPY =  0.19070900442001654
RIGHT NODE WITH ENTROPY =  0.17556502585750278

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.031817098484323514
SPLITTED AS ejectio


FEATURE : age
MAXIMUM INFORMATION GAINED : 0.5435644431995964
SPLITTED AS age < 55.0 AND age >= 55.0
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : serum_sodium
MAXIMUM INFORMATION GAINED : 0.1243449875585938
SPLITTED AS serum_sodium < 139.5 AND serum_sodium >= 139.5
LEFT NODE WITH ENTROPY =  0.5746356978376794
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.1752941647252571
SPLITTED AS time < 220.0 AND time >= 220.0
LEFT NODE WITH ENTROPY =  0.4226707635691618
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : sex
MAXIMUM INFORMATION GAINED : 0.06161659794782448
SPLITTED AS sex < 0.5 AND sex >= 0.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.5772899021747756

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.2999725229735489
SPLITTED AS ejection_fraction < 42.5 AND ejection_fraction >= 42.5
LEFT NODE WITH ENTROPY =  0.3040467651483329
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.0768

In [24]:
#predict and store the results in a list called predictions
predictions_random_forest = []
for index,rows in test_df.iterrows():
    result = predict_using_random_forest(rows,heart_disease_decision_tree_model)
#     print("result ",result)
    predictions_random_forest.append(result)

In [25]:
#calculate the accuracy of the dataset
actual_values = test_df["DEATH_EVENT"].reset_index(drop=True)
acc_of_descision_tree = accuracy(predictions_random_forest,actual_values)

print("\nACCURACY USING RANDOM FOREST : ",acc_of_descision_tree ," percentage")

predictions actual values

0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
1   1
0   0
0   0
0   0
0   0
0   0
0   0
1   1
0   0
1   1
0   0
1   1
0   0
1   1
0   0
1   1
1   1
0   0
0   0
0   0
1   1
0   0
0   0
0   0
1   1
0   0
0   0
0   0
1   1
1   1
0   0
1   1
1   1
1   1
0   0
1   1
0   0
0   0
0   0
1   1
0   0
0   0
0   0
0   0
0   0
1   1
0   0
1   1
0   0
0   0
0   0
1   1
0   0
1   1
0   0
0   0
0   0
0   0
0   0
0   0
1   1
1   1
0   0
0   0
0   0
1   1
0   0
0   0
1   1
0   0
1   1
0   0
1   1
0   0
1   1
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
1   1
0   0
1   1
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
1   1
0   0
0   0
1   1
0   0
0   0
1   1
0   0
0   0
1   1
0   0
0   0
0   0
0   0
1   1
0   0
0   0
1   1
0   0
1   1
0   0
0   0
1   1
0   0
1   1
0   0
1   1
0   0
0   0
0   0
0   0
1   1
0   0
0   0
1   1
0   0
0   0
0   0
0   0
1   1
0   0
0   0
0   0
0   0
1   1
0   0
1   1
0   0
0   0
0   0
1

PREDICTION USING DECISION TREE

In [26]:
#training a decision tree with maximum depth of 100 and maximum leaf size of 10
decision_tree_model = decision_tree_fit(dataset,0,100,10)


FEATURE : time
MAXIMUM INFORMATION GAINED : 0.3018804543423492
SPLITTED AS time < 67.5 AND time >= 67.5
LEFT NODE WITH ENTROPY =  0.5401172058446086
RIGHT NODE WITH ENTROPY =  0.6114194578284544

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.06386899482663072
SPLITTED AS age < 61.0 AND age >= 61.0
LEFT NODE WITH ENTROPY =  0.7687677679419425
RIGHT NODE WITH ENTROPY =  0.21943756321683852

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.10589591363776507
SPLITTED AS ejection_fraction < 27.5 AND ejection_fraction >= 27.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.8788073826002352

FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.1369588504063861
SPLITTED AS ejection_fraction < 32.5 AND ejection_fraction >= 32.5
LEFT NODE WITH ENTROPY =  0.9527431707334315
RIGHT NODE WITH ENTROPY =  0.6607352096786251

FEATURE : platelets
MAXIMUM INFORMATION GAINED : 0.6602634813663066
SPLITTED AS platelets < 298500.0 AND platelets >= 298500.0
LEFT NODE WITH ENTROPY = 


FEATURE : creatinine_phosphokinase
MAXIMUM INFORMATION GAINED : 0.11283843477865763
SPLITTED AS creatinine_phosphokinase < 57.5 AND creatinine_phosphokinase >= 57.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.1623261801753929

FEATURE : creatinine_phosphokinase
MAXIMUM INFORMATION GAINED : 0.06936179418596555
SPLITTED AS creatinine_phosphokinase < 1918.5 AND creatinine_phosphokinase >= 1918.5
LEFT NODE WITH ENTROPY =  0.09408443883267346
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : diabetes
MAXIMUM INFORMATION GAINED : 0.09408443883267346
SPLITTED AS diabetes < 0.5 AND diabetes >= 0.5
LEFT NODE WITH ENTROPY =  0.0
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.04717717857918763
SPLITTED AS age < 79.5 AND age >= 79.5
LEFT NODE WITH ENTROPY =  0.1927130347896255
RIGHT NODE WITH ENTROPY =  0.9919924034538556

FEATURE : creatinine_phosphokinase
MAXIMUM INFORMATION GAINED : 0.03566425020164851
SPLITTED AS creatinine_phosphokinase < 2307.5 AND creat


FEATURE : ejection_fraction
MAXIMUM INFORMATION GAINED : 0.1574875577192607
SPLITTED AS ejection_fraction < 32.5 AND ejection_fraction >= 32.5
LEFT NODE WITH ENTROPY =  0.8952722642402939
RIGHT NODE WITH ENTROPY =  0.7785391795129595

FEATURE : time
MAXIMUM INFORMATION GAINED : 0.15784465166005324
SPLITTED AS time < 242.5 AND time >= 242.5
LEFT NODE WITH ENTROPY =  0.7968174923244051
RIGHT NODE WITH ENTROPY =  0.18312206830137276

FEATURE : serum_creatinine
MAXIMUM INFORMATION GAINED : 0.16721122007326583
SPLITTED AS serum_creatinine < 3.2 AND serum_creatinine >= 3.2
LEFT NODE WITH ENTROPY =  0.6802177089272758
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.10128287747121945
SPLITTED AS age < 70.0 AND age >= 70.0
LEFT NODE WITH ENTROPY =  0.8184033299219706
RIGHT NODE WITH ENTROPY =  0.0

FEATURE : age
MAXIMUM INFORMATION GAINED : 0.30593276351574816
SPLITTED AS age < 60.8335 AND age >= 60.8335
LEFT NODE WITH ENTROPY =  0.5655511182519259
RIGHT NODE WITH 

In [27]:
predictions_decision_tree = []
for index,rows in test_df.iterrows():
    result = predict_using_decision_tree(rows,decision_tree_model)
    #print("result ",result)
    predictions_decision_tree.append(result)

In [28]:
#calculate the accuracy of the dataset
actual_values = test_df["DEATH_EVENT"].reset_index(drop=True)
acc_of_decision_tree = accuracy(predictions_decision_tree,actual_values)

print("\nACCURACY OF DECISION TREE : ",acc_of_decision_tree ," percentage")

predictions actual values

0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
1   1
0   0
0   0
0   0
0   0
0   0
0   0
1   1
0   0
1   1
0   0
1   1
0   0
1   1
0   0
1   1
1   1
0   0
0   0
0   0
1   1
0   0
0   0
0   0
1   1
0   0
0   0
0   0
1   1
1   1
0   0
1   1
1   1
1   1
0   0
1   1
0   0
0   0
0   0
1   1
0   0
0   0
0   0
0   0
0   0
1   1
0   0
1   1
0   0
0   0
0   0
1   1
0   0
1   1
0   0
0   0
0   0
0   0
0   0
1   0
1   1
1   1
0   0
0   0
0   0
1   1
0   0
0   0
1   1
0   0
1   1
0   0
1   1
0   0
1   1
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
1   1
0   0
1   1
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
1   1
0   0
0   0
1   1
0   0
0   0
1   1
0   0
0   0
1   1
0   0
0   0
0   0
0   0
1   1
0   0
0   0
1   1
0   0
1   1
0   0
0   0
1   1
0   0
1   1
0   0
1   1
0   0
0   0
0   0
0   0
1   1
0   0
0   0
1   1
0   0
0   0
0   0
0   0
1   1
0   0
0   0
0   0
0   0
1   1
0   0
1   1
0   0
0   0
0   0
1

In [29]:
print("\nACCURACY OF DECISION TREE : ",acc_of_decision_tree ," percentage")
print("\nACCURACY USING RANDOM FOREST : ",acc_of_descision_tree ," percentage")


ACCURACY OF DECISION TREE :  99.7  percentage

ACCURACY USING RANDOM FOREST :  100.0  percentage
